<a href="https://colab.research.google.com/github/oraziotorre/TTinsight/blob/main/TTinsight_DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TTinsight**

---


## Data Cleaning & Feature Engineering

In questo notebook vedremo tutto il processo per il caricamento, l'analisi e la pulizia dei dati. Sceglieremo poi le feature migliori e generaremo il dataset finale.

## **Inizializzazione**

In [84]:
# Pandas per la gestione dei dati
import pandas as pd
import ast

# Per visualizzare i DataFrame su Google Colab in formato tabellare
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [85]:
# Importiamo il dataset ottenuto tramite le operazioni di scraping
dataset = pd.read_csv("raw_dataset.csv")

dataset

,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,points_progression,opponent_points
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ..."
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-1,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ..."
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,1-1,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ..."
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,2-1,"[1, 1, 1, 2, 3, 3, 4, 4, 4, 5, 5, 6, 6, 6, 6, ...","[0, 1, 2, 2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 8, 9, ..."
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,2-2,"[1, 1, 2, 2, 3, 3, 4, 4, 4, 5, 6, 6, 6, 7, 7, ...","[0, 1, 1, 2, 2, 3, 3, 4, 5, 5, 5, 6, 7, 7, 8, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72092,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,0-0,"[1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3]","[0, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9, 10]"
72093,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,0-1,"[0, 1, 2, 2, 2, 3, 3, 4, 5, 6, 7, 7, 7, 8, 8, ...","[1, 1, 1, 2, 3, 3, 4, 4, 4, 4, 4, 5, 6, 6, 7, ..."
72094,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-1,"[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...","[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ..."
72095,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-2,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ..."


## **Etichettatura**
Aggiungiamo al dataset una nuova colonna, denominata `set_result`, che indicherà il risultato di ciascun set per ogni partita:

- `1` per identificare la classe dei Vincitori del set.
- `0` per identificare la classe degli Sconfitti del set.

Questa etichettatura risulta particolarmente utile per l'addestramento di modelli come LSTM e Regressione Logistica, in cui l'obiettivo è prevedere, con una determinata probabilità, se un giocatore vincerà o perderà il set in base alle altre caratteristiche presenti nel dataset.

In [86]:
# Funzione per convertire i punteggi da stringa a lista di interi
def convert_to_int_list(points_str):
    if isinstance(points_str, str):
        return list(map(int, points_str.strip('[]').split(', ')))
    else:
        return None


# Funzione per determinare il risultato del set
def calculate_set_result(points_progression, opponent_points):
    if points_progression:
        if points_progression[-1] > opponent_points[-1]:
            return 1  # Vittoria per il giocatore
        else:
            return 0  # Sconfitta per il giocatore
    else:
        return -1  # Dati non conformi


set_results = []

dataset['points_progression'] = dataset['points_progression'].apply(convert_to_int_list)
dataset['opponent_points'] = dataset['opponent_points'].apply(convert_to_int_list)

for index, row in dataset.iterrows():
    result = calculate_set_result(row['points_progression'], row['opponent_points'])
    set_results.append(result)

# Aggiungo la nuova colonna 'set_result' al DataFrame
dataset['set_result'] = set_results

dataset




,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,points_progression,opponent_points,set_result
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-1,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",1
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,1-1,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ...",1
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,2-1,"[1, 1, 1, 2, 3, 3, 4, 4, 4, 5, 5, 6, 6, 6, 6, ...","[0, 1, 2, 2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 8, 9, ...",0
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,2-2,"[1, 1, 2, 2, 3, 3, 4, 4, 4, 5, 6, 6, 6, 7, 7, ...","[0, 1, 1, 2, 2, 3, 3, 4, 5, 5, 5, 6, 7, 7, 8, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72092,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,0-0,"[1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3]","[0, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9, 10]",0
72093,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,0-1,"[0, 1, 2, 2, 2, 3, 3, 4, 5, 6, 7, 7, 7, 8, 8, ...","[1, 1, 1, 2, 3, 3, 4, 4, 4, 4, 4, 5, 6, 6, 7, ...",1
72094,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-1,"[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...","[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...",0
72095,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-2,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1


## **Analisi dei dati**

Esaminiamo la qualità e la correttezza dei dati, valutandone l'affidabilità e identificando eventuali incongruenze o anomalie che potrebbero influenzare l'addestramento.

### **Controllo del bilanciamento delle classi**
Un primo passo nell'analisi del dataset consiste nel verificare se le classi presenti siano bilanciate.

Per farlo, analizziamo la distribuzione delle istanze con valore `1` e `0` nella colonna `set_result`.

In [87]:
# Numero di elementi per la classe "Vincitori"
print(len(dataset[(dataset['set_result'] == 1)]))
# Numero di elementi per la classe "Sconfitti"
print(len(dataset[(dataset['set_result'] == 0)]))

# Totale
print(len(dataset))


35234
34373
72097


Notiamo che abbiamo un dataset quasi perfettamente bilanciato e di conseguenza non sono necessari ulteriori accorgimenti



### **Analisi delle caratteristiche e delle distribuzioni dei dati**
Usiamo `describe()` per avere una overview sulle distribuzioni dei dati numerici

In [88]:
# Verifico la correttezza dei campi numerici
dataset.describe()

,event_id,stage_id,match_duration,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,sets_required_to_win,set_result
count,72097.000000,72097.000000,72097.000000,72097.000000,17558.000000,72097.000000,17558.000000,72097.000000,72097.000000,72097.000000,72097.000000
mean,2702.487149,1556.146095,2058.918221,123932.242964,125566.084919,123741.393595,126035.559745,2.107397,2.066202,3.146705,0.454166
std,156.934388,5453.980041,1575.258589,17247.564085,17316.489865,17089.894592,18276.388290,1.245484,1.255210,0.353971,0.563007
min,2234.000000,100.000000,0.000000,100001.000000,100001.000000,90051.000000,100032.000000,0.000000,0.000000,2.000000,-1.000000
25%,2569.000000,200.000000,1426.000000,114716.000000,115754.000000,114715.000000,115548.000000,1.000000,1.000000,3.000000,0.000000
50%,2701.000000,600.000000,1914.000000,121408.000000,122421.000000,121103.000000,122428.000000,3.000000,3.000000,3.000000,0.000000
75%,2869.000000,1200.000000,2430.000000,132132.000000,133124.000000,132022.000000,133124.000000,3.000000,3.000000,3.000000,1.000000
max,2997.000000,110005.000000,77074.000000,212088.000000,212090.000000,211762.000000,211733.000000,5.000000,4.000000,5.000000,1.000000


Notiamo che nella colonna `set_required_to_win`, si riscontrano dei valori rari (`2` e `5`)  che potrebbero non essere rilevanti per il nostro obiettivo di analisi.

Questi valori potrebbero introdurre rumore o distorcere le inferenze, soprattutto perché la loro frequenza è estremamente bassa rispetto agli altri valori.

### **Controllo dei valori nulli**

Verificare la presenza di valori nulli all'interno del dataset.

In [89]:
# Identificare i valori NaN nel dataset
nan_mask = dataset.isna()

# Conta i valori NaN (valori mancanti) per ciascuna colonna del dataset
nan_count = nan_mask.sum()

# Verifico i valori null del raw_dataset
nan_count


,0
event_id,0
match_id,0
match_format,0
players_gender,0
match_stage,0
stage_id,0
match_duration,0
match_start_time,0
player_id,0
player_2_id,54539


Le colonne `points_progression` e `opponent_points` contengono **2490** valori nulli, che richiedono un intervento per la gestione.

Le colonne `player_2_id` e `opponent_2_id` presentano invece **54539** valori nulli, ma ciò è normale, in quanto queste colonne riguardano il secondo giocatore di una squadra, quindi sono nulle nelle partite di singolo.



## **Operazione sulle feature**


Facciamo alcune considerazioni sulle caratteristiche del nostro dataset e identifichiamo possibili miglioramenti per ottimizzarlo in vista del prossimo training del modello:

- Eliminazione delle righe in cui sono stati trovati valori nulli e indesiderati nelle colonna `points_progression` e `sets_required_to_win` individuati in fase di analisi

- Le colonne `sets_required_to_win` e `current_match_state` verranno sostituite con due nuove colonne: `final_set_a` e `final_set_b` per rendere il dataset più chiaro e organizzato

- Trasformazione dei valori T nella colonna `match_format` in formati che siano o S o D

- Operazioni sulla colonna `points_progression`, fondamentale per il successivo addestramento dei modelli


-  Le colonne `event_id`,`match_id`, `stage_id`, `match_duration`, `players_gender`, `match_start_time`, `opponent_id`, `opponent_2_id`,`player_sets_won`, `opponent_sets_won`, `match_scores`, `opponent_points`,`match_format`, `player_id`, `player_2_id`, `sets_required_to_win`, `current_match_state` non sono necessarie per il training dei modelli e possono essere cancellate




### **Gestione dei valori nulli e indesiderati**
Dall'analisi precedente, abbiamo constatato che la feature `points_progression` manca di alcuni suoi valori. La strategia che abbiamo applicato è l'eliminazione delle righe corrispondenti.

In [90]:
# Rimuoviamo tutte le righe del dataset in cui la colonna 'points_progression' contiene valori NaN
dataset = dataset.dropna(subset=["points_progression"])

# Verifichiamo che non esistano più righe con valori NaN nella colonna 'points_progression'
dataset[dataset['points_progression'].isnull()]

,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,points_progression,opponent_points,set_result


In base a ciò che abbiamo appreso precedentemente da `describe()`, rimuoviamo i le righe non desiderate con valore della colonna `sets_required_to_win` uguale a `2` o `5`.

Questi valori rari potrebbero rappresentare formati di gioco non standard (es. partite amichevoli o partite abbreviate per motivi eccezionali), non rilevanti per l'analisi



In [91]:
# Rimuoviamo le righe contenenti set con punteggi rari che non vogliamo considerare
dataset = dataset[~dataset['sets_required_to_win'].isin([2, 5])]

dataset.describe()

,event_id,stage_id,match_duration,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,sets_required_to_win,set_result
count,69604.000000,69604.000000,69604.000000,69604.000000,16877.000000,69604.000000,16877.000000,69604.000000,69604.000000,69604.000000,69604.000000
mean,2702.922073,1549.569867,2045.960505,123927.682705,125527.565029,123718.854764,125999.362446,2.104735,2.065485,3.146328,0.506207
std,156.757815,5440.504349,1410.742190,17268.757266,17254.804727,17071.963284,18221.092496,1.246822,1.256013,0.353437,0.499965
min,2234.000000,100.000000,0.000000,100001.000000,100001.000000,90051.000000,100032.000000,0.000000,0.000000,3.000000,0.000000
25%,2569.000000,200.000000,1422.000000,114716.000000,115565.000000,114715.000000,115548.000000,1.000000,1.000000,3.000000,0.000000
50%,2701.000000,600.000000,1912.000000,121404.000000,122419.000000,121074.000000,122428.000000,3.000000,3.000000,3.000000,1.000000
75%,2869.000000,1200.000000,2427.000000,132132.000000,133124.000000,132022.000000,133124.000000,3.000000,3.000000,3.000000,1.000000
max,2997.000000,110005.000000,77074.000000,212088.000000,212090.000000,211762.000000,211733.000000,4.000000,4.000000,4.000000,1.000000


###**Colonne *sets_required_to_win* e *current_match_state***

Le colonne `sets_required_to_win` e `current_match_state` fanno entrambe riferimento al numero di set necessari per vincere una partita. Per rendere il dataset più chiaro e organizzato, possiamo sostituirle con due nuove colonne: `final_set_a` e `final_set_b`, che rappresentano rispettivamente un booleano che indica il set finale per il **Giocatore A** e per l'avversario **Giocatore B** necessario a vincere la partita.

Determiniamo se un giocatore o il suo avversario è a un set dalla vittoria
- Se il giocatore è un set dalla vittoria poniamo `final_set_a` pari a `1`
- Se l'avversario è un set dalla vittoria poniamo `final_set_b` pari a `1`
- Altrimenti sarà pari a `0`

In [92]:
final_set_a_diff = dataset['sets_required_to_win'] - dataset['current_match_state'].str.split('-', expand=True)[0].astype(int)
final_set_b_diff = dataset['sets_required_to_win'] - dataset['current_match_state'].str.split('-', expand=True)[1].astype(int)

# Crea una colonna 'final_set_a' che vale 1 se il giocatore è a un set dalla vittoria, altrimenti 0
dataset['final_set_a'] = (final_set_a_diff == 1).astype(int)

# Crea una colonna 'final_set_b' che vale 1 se l'avversario è a un set dalla vittoria, altrimenti 0
dataset['final_set_b'] = (final_set_b_diff == 1).astype(int)

# Riposizionamento delle nuove colonne
is_final_index = dataset.columns.get_loc('current_match_state')
final_set_a_column = dataset.pop('final_set_a')
final_set_b_column = dataset.pop('final_set_b')
dataset.insert(is_final_index + 1, 'final_set_a', final_set_a_column)
dataset.insert(is_final_index + 2, 'final_set_b', final_set_b_column)

dataset


,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,...,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,final_set_a,final_set_b,points_progression,opponent_points,set_result
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,0,0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-1,0,0,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",1
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,1-1,0,0,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ...",1
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,2-1,1,0,"[1, 1, 1, 2, 3, 3, 4, 4, 4, 5, 5, 6, 6, 6, 6, ...","[0, 1, 2, 2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 8, 9, ...",0
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,2-2,1,1,"[1, 1, 2, 2, 3, 3, 4, 4, 4, 5, 6, 6, 6, 7, 7, ...","[0, 1, 1, 2, 2, 3, 3, 4, 5, 5, 5, 6, 7, 7, 8, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72092,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,2,3,"3-11,11-7,9-11,11-7,6-11",3,0-0,0,0,"[1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3]","[0, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9, 10]",0
72093,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,2,3,"3-11,11-7,9-11,11-7,6-11",3,0-1,0,0,"[0, 1, 2, 2, 2, 3, 3, 4, 5, 6, 7, 7, 7, 8, 8, ...","[1, 1, 1, 2, 3, 3, 4, 4, 4, 4, 4, 5, 6, 6, 7, ...",1
72094,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-1,0,0,"[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...","[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...",0
72095,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-2,0,1,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1


### **Colonna *match_format***

Modifichiamo la colonna `match_format`, sostituendo tutti i valori `T` (partita di team) con `S` (partita di singolo) o `D` (partita di doppio)

Viene eseguita questa operazione per **evitare la succesiva rimozione di istanze** che potrebbero ritornarci utili nello sviluppo successivo dei nostri modelli.

In [93]:
# Funzione per determinare il match_format quando è Team (Singolo o Doppio)
def determine_match_format(player_2_id):
    if pd.isna(player_2_id):
        return 'S'  # Ritorna 'S' per indicare una partita singola
    else:
        return 'D'  # Ritorna 'D' per indicare una partita doppia


dataset['match_format'] = dataset['player_2_id'].apply(determine_match_format)

dataset


,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,...,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,final_set_a,final_set_b,points_progression,opponent_points,set_result
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,0,0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-1,0,0,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",1
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,1-1,0,0,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ...",1
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,2-1,1,0,"[1, 1, 1, 2, 3, 3, 4, 4, 4, 5, 5, 6, 6, 6, 6, ...","[0, 1, 2, 2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 8, 9, ...",0
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,2-2,1,1,"[1, 1, 2, 2, 3, 3, 4, 4, 4, 5, 6, 6, 6, 7, 7, ...","[0, 1, 1, 2, 2, 3, 3, 4, 5, 5, 5, 6, 7, 7, 8, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72092,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,2,3,"3-11,11-7,9-11,11-7,6-11",3,0-0,0,0,"[1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3]","[0, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9, 10]",0
72093,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,2,3,"3-11,11-7,9-11,11-7,6-11",3,0-1,0,0,"[0, 1, 2, 2, 2, 3, 3, 4, 5, 6, 7, 7, 7, 8, 8, ...","[1, 1, 1, 2, 3, 3, 4, 4, 4, 4, 4, 5, 6, 6, 7, ...",1
72094,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-1,0,0,"[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...","[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...",0
72095,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-2,0,1,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1


### **Colonna *points_progression***

#### Trasformazione del formato
Ogni istanza della colonna viene convertita in un vettore binario, dove ogni elemento rappresenta un punto nel set. Il valore del vettore sarà 1 se il giocatore ha fatto un punto in quel particolare istante del set, e 0 altrimenti. Questo formato rende il vettore della colonna `points_progression` più maneggevole e adatto all'addestramento dei modelli.

In [94]:
def transform_points_progression(points):
    transformed = []

    for i in range(len(points)):
        points[i] = int(points[i])

        # Per il primo punteggio, aggiungiamo 1 se il punteggio è maggiore di 0, altrimenti 0
        if i == 0:
            transformed.append(1 if points[i] > 0 else 0)
        else:
            transformed.append(1 if points[i] > points[i - 1] else 0)

    return transformed

dataset['points_progression'] = dataset['points_progression'].apply(transform_points_progression)

dataset


,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,...,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,final_set_a,final_set_b,points_progression,opponent_points,set_result
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,0,0,"[0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-1,0,0,"[1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",1
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,1-1,0,0,"[1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ...",1
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,2-1,1,0,"[1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, ...","[0, 1, 2, 2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 8, 9, ...",0
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,2-2,1,1,"[1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, ...","[0, 1, 1, 2, 2, 3, 3, 4, 5, 5, 5, 6, 7, 7, 8, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72092,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,2,3,"3-11,11-7,9-11,11-7,6-11",3,0-0,0,0,"[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]","[0, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9, 10]",0
72093,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,2,3,"3-11,11-7,9-11,11-7,6-11",3,0-1,0,0,"[0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, ...","[1, 1, 1, 2, 3, 3, 4, 4, 4, 4, 4, 5, 6, 6, 7, ...",1
72094,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-1,0,0,"[1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, ...","[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...",0
72095,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-2,0,1,"[1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1


#### Generazione delle sottosequenze di punteggio
Per ogni istanza di un set con punteggio finale (ad esempio, 10-5), la funzione crea tutte le possibili situazioni intermedie dei punteggi. Questo significa che, dato il punteggio finale del set, vengono estratte tutte le progressioni dei punteggi intermedi (ad esempio, 1-0, 2-0, 3-1, e così via) fino al punteggio finale. In pratica, la funzione analizza la sequenza di punti per ricostruire ogni stato intermedio del set, consentendo una visione più dettagliata dell'andamento della partita e della sua evoluzione.

In [95]:
def generate_score_sequence(points_progression):

    if isinstance(points_progression, str):
      points_progression = ast.literal_eval(points_progression)

    if isinstance(points_progression, list):
        points_progression = [int(x) for x in points_progression]

    # Crea le sottosequenze
    sequence = [list(points_progression[:i]) for i in range(1, len(points_progression) + 1)]

    return sequence


dataset['points_progression'] = dataset['points_progression'].apply(generate_score_sequence)
dataset = dataset.explode('points_progression', ignore_index=True)

dataset

,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,...,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,final_set_a,final_set_b,points_progression,opponent_points,set_result
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,0,0,[0],"[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,0,0,"[0, 0]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,0,0,"[0, 0, 1]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,0,0,"[0, 0, 1, 1]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,0,0,"[0, 0, 1, 1, 0]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207431,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,2,3,"3-11,11-7,9-11,11-7,6-11",3,2-2,1,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0
1207432,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,2,3,"3-11,11-7,9-11,11-7,6-11",3,2-2,1,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0
1207433,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,2,3,"3-11,11-7,9-11,11-7,6-11",3,2-2,1,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0
1207434,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,2,3,"3-11,11-7,9-11,11-7,6-11",3,2-2,1,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0


#### Rimozione delle sottosequenze indesiderate
Filtriamo le sequenze di punteggi non rilevanti per il nostro scopo:

- **Sequenze che arrivano ai vantaggi**: Prevedere il vincitore diventa troppo complesso a causa delle numerose combinazioni possibili, per cui queste sequenze vengono escluse.

- **Sequenze che arrivano al set point**: Quando un giocatore raggiunge il set point senza vantaggi, l'esito è praticamente certo (chi ha 10 punti è quasi sempre il vincitore), rendendo la previsione superflua.

- **Situazioni parziali del set con meno di 6 punti giocati**: Sequenze con pochi punti non forniscono un contesto sufficiente per una previsione accurata e quindi vengono eliminate.

- **Situazioni con una differenza di punteggio maggiore di 3 punti**: Quando la differenza tra i punteggi è troppo ampia, il risultato è altamente prevedibile, rendendo queste sequenze meno interessanti o informative.

Questi filtri sono applicati per ridurre la dimensionalità del dataset e concentrare i modelli sulle situazioni più significative e complesse da analizzare.

In [96]:
def remove_rows_with_large_points_progression(df):

    def is_valid_progression(points_progression):

        if isinstance(points_progression, str):
            points_progression = ast.literal_eval(points_progression)

        # Conta gli 0 e gli 1
        count_zeros = points_progression.count(0)
        count_ones = points_progression.count(1)
        total_points = count_zeros + count_ones

        # Applica i filtri
        return (
            6 < total_points < 20 and
            count_zeros < 10 and
            count_ones < 10 and
            abs(count_zeros - count_ones) < 4
        )

    # Filtra le righe
    return df[df['points_progression'].apply(is_valid_progression)]


dataset = remove_rows_with_large_points_progression(dataset)
dataset

,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,...,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,final_set_a,final_set_b,points_progression,opponent_points,set_result
6,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,0,0,"[0, 0, 1, 1, 0, 1, 1]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
7,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,0,0,"[0, 0, 1, 1, 0, 1, 1, 0]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
8,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,0,0,"[0, 0, 1, 1, 0, 1, 1, 0, 0]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
9,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,0,0,"[0, 0, 1, 1, 0, 1, 1, 0, 0, 1]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
10,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,0,0,"[0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207428,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,2,3,"3-11,11-7,9-11,11-7,6-11",3,2-2,1,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0
1207430,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,2,3,"3-11,11-7,9-11,11-7,6-11",3,2-2,1,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0
1207431,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,2,3,"3-11,11-7,9-11,11-7,6-11",3,2-2,1,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0
1207432,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,2,3,"3-11,11-7,9-11,11-7,6-11",3,2-2,1,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",0


#### Aggiunta di feature derivate
In base ai valori della colonna `points_progression`, vengono aggiunte nuove feature per arricchire il dataset e renderlo più adatto all'addestramento di modelli che non possono elaborare direttamente sequenze di punteggi. Le nuove feature includono:

- `score_a`: Rappresenta il punteggio accumulato dal giocatore A.

- `score_b`: Rappresenta il punteggio accumulato dal giocatore B.

- `half_diff`: Rappresenta la differenza di punti tra i due giocatori considerando solo l'ultima metà della sequenza di punteggi.

- `third_diff`: Rappresenta la differenza di punti tra i due giocatori considerando solo l'ultimo terzo della sequenza di punteggi.

- `comeback`: Rappresenta il numero attuale di punti consecutivi realizzati dal giocatore in svantaggio, calcolato in funzione dei punti fatti finora. Il valore sarà negativo se il giocatore che sta effettuando il comeback è l'avversario.

- `length`: Rappresenta il numero di punti giocati nell'istanza in considerazione

Queste nuove feature sono introdotte per trasformare le sequenze in rappresentazioni numeriche statiche, rendendole adatte a modelli di machine learning tradizionali.

In [97]:
def aggiungi_features(dataset):

    score_a = []
    score_b = []
    half_diff = []
    third_diff = []
    comeback = []
    length = []

    for points in dataset['points_progression']:

        punti_a = punti_b = rimonta_a = rimonta_b = 0

        for point in points:
            if point == 1:
                rimonta_a += 1
                rimonta_b = 0
                punti_a += 1
            elif point == 0:
                rimonta_b += 1
                rimonta_a = 0
                punti_b += 1


        # Aggiungo 'score_a'
        score_a.append(punti_a)

        # Aggiungo 'score_b'
        score_b.append(punti_b)


        # Aggiungo 'comeback'
        if rimonta_a > rimonta_b:
            comeback.append(rimonta_a / punti_a)
        else:
            comeback.append(-rimonta_b / punti_b)



        punti_a = punti_b = 0
        start_index = len(points) // 2

        for point in points[start_index:]:
            if point == 1:
                punti_a += 1
            elif point == 0:
                punti_b += 1

        # Aggiungo 'half_diff'
        half_diff.append(punti_a - punti_b)


        punti_a = punti_b = 0
        start_index = 2*(len(points) // 3)+1

        for point in points[start_index:]:
            if point == 1:
                punti_a += 1
            elif point == 0:
                punti_b += 1

        # Aggiungo 'third_diff'
        third_diff.append(punti_a - punti_b)

        # Aggiungo 'length'
        length.append(len(points))


    # Aggiunge le nuove feature al dataset
    dataset = dataset.assign(
        score_a=score_a,
        score_b=score_b,
        comeback=comeback,
        half_diff=half_diff,
        third_diff=third_diff,
        length=length
    )

    return dataset

dataset = aggiungi_features(dataset)
columns = dataset.columns.tolist()
columns.remove('set_result')
columns.append('set_result')
dataset = dataset[columns]
dataset

,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,...,final_set_b,points_progression,opponent_points,score_a,score_b,comeback,half_diff,third_diff,length,set_result
6,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,0,"[0, 0, 1, 1, 0, 1, 1]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",4,3,0.500000,2,2,7,0
7,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,0,"[0, 0, 1, 1, 0, 1, 1, 0]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",4,4,-0.250000,0,1,8,0
8,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,0,"[0, 0, 1, 1, 0, 1, 1, 0, 0]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",4,5,-0.400000,-1,-2,9,0
9,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,0,"[0, 0, 1, 1, 0, 1, 1, 0, 0, 1]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",5,5,0.200000,1,-1,10,0
10,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,...,0,"[0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1]","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",6,5,0.333333,2,0,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207428,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",3,6,-0.833333,-5,-2,9,0
1207430,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",4,7,0.250000,-4,-2,11,0
1207431,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",5,7,0.400000,-2,1,12,0
1207432,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",5,8,-0.125000,-3,0,13,0



### **Operazione su *single_matches_dataset***

Attraverso l'analisi della colonna `match_format` generiamo un nuovo dataset contente solo le partite di singolo.

Questo dataset potrà essere successivamente utile nel testing dei modelli.

In [98]:
# Rimuoviamo dal dataset le partite di doppio
rows_to_remove = dataset[dataset['match_format'] == 'D'].index

dataset_singles = dataset.drop(index=rows_to_remove)

dataset_singles

,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,...,final_set_b,points_progression,opponent_points,score_a,score_b,comeback,half_diff,third_diff,length,set_result
132,2234,TTEMTEAM--------------FNL-00010001--------,S,M,FNL,10001,2387,2021-11-19 23:25:33,111683,NaN,...,0,"[0, 1, 1, 0, 0, 1, 0]","[1, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 7, 8, ...",3,4,-0.250000,-2,0,7,1
133,2234,TTEMTEAM--------------FNL-00010001--------,S,M,FNL,10001,2387,2021-11-19 23:25:33,111683,NaN,...,0,"[0, 1, 1, 0, 0, 1, 0, 1]","[1, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 7, 8, ...",4,4,0.250000,0,1,8,1
134,2234,TTEMTEAM--------------FNL-00010001--------,S,M,FNL,10001,2387,2021-11-19 23:25:33,111683,NaN,...,0,"[0, 1, 1, 0, 0, 1, 0, 1, 0]","[1, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 7, 8, ...",4,5,-0.200000,-1,0,9,1
135,2234,TTEMTEAM--------------FNL-00010001--------,S,M,FNL,10001,2387,2021-11-19 23:25:33,111683,NaN,...,0,"[0, 1, 1, 0, 0, 1, 0, 1, 0, 1]","[1, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 7, 8, ...",5,5,0.200000,1,1,10,1
136,2234,TTEMTEAM--------------FNL-00010001--------,S,M,FNL,10001,2387,2021-11-19 23:25:33,111683,NaN,...,0,"[0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1]","[1, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 7, 8, ...",6,5,0.333333,2,2,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207428,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",3,6,-0.833333,-5,-2,9,0
1207430,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",4,7,0.250000,-4,-2,11,0
1207431,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",5,7,0.400000,-2,1,12,0
1207432,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,...,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",5,8,-0.125000,-3,0,13,0


### **Eliminazione delle colonne non adatte all'addestramento**

Vantaggi di questa operazione:
- **Eliminando colonne non necessarie**, il dataset diventa più semplice e focalizzato sui dati utili.
- Rimuovendo informazioni irrilevanti, l'addestramento risulta più chiaro e veloce, **evitando di appesantire** il modello con dati superflui.

In [100]:
# Definiamo le colonne da eliminare
columns_to_drop = ['event_id', 'match_id', 'stage_id', 'match_duration','match_stage',
                   'players_gender','match_start_time','opponent_id', 'opponent_2_id',
                  'player_sets_won', 'opponent_sets_won', 'match_scores',
                  'opponent_points','match_format', 'player_id', 'player_2_id',
                   'sets_required_to_win', 'current_match_state']

# Rimuovo le colonne che non mi interessano
dataset = dataset.drop(columns=columns_to_drop, axis=1)
dataset_singles = dataset_singles.drop(columns=columns_to_drop, axis=1)

dataset_singles

,final_set_a,final_set_b,points_progression,score_a,score_b,comeback,half_diff,third_diff,length,set_result
132,0,0,"[0, 1, 1, 0, 0, 1, 0]",3,4,-0.250000,-2,0,7,1
133,0,0,"[0, 1, 1, 0, 0, 1, 0, 1]",4,4,0.250000,0,1,8,1
134,0,0,"[0, 1, 1, 0, 0, 1, 0, 1, 0]",4,5,-0.200000,-1,0,9,1
135,0,0,"[0, 1, 1, 0, 0, 1, 0, 1, 0, 1]",5,5,0.200000,1,1,10,1
136,0,0,"[0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1]",6,5,0.333333,2,2,11,1
...,...,...,...,...,...,...,...,...,...,...
1207428,1,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0]",3,6,-0.833333,-5,-2,9,0
1207430,1,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1]",4,7,0.250000,-4,-2,11,0
1207431,1,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1]",5,7,0.400000,-2,1,12,0
1207432,1,1,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0]",5,8,-0.125000,-3,0,13,0


### **Salvataggio dei nuovi dataset**

In [101]:
# Salviamo il dataset pulito dai valori indesiderati
dataset.to_csv("cleaned_dataset.csv", index=False)

In [102]:
# Salviamo il dataset finale delle partite di singolo
dataset_singles.to_csv("singles_matches_dataset.csv", index=False)